In [20]:
import random
import math
from typing import List, Tuple, Optional

Clause = List[int]
CNF = List[Clause]


def evaluate_clause(clause: Clause, assignment: List[bool]) -> bool:
    for lit in clause:
        var = abs(lit) - 1
        val = assignment[var]
        if lit < 0:
            val = not val
        if val:
            return True
    return False


def count_unsat(cnf: CNF, assignment: List[bool]) -> int:
    return sum(0 if evaluate_clause(c, assignment) else 1 for c in cnf)


def build_occurrence_lists(cnf: CNF, n_vars: int) -> List[List[int]]:
    """
    Per ogni variabile, lista degli indici di clausole che la contengono (in qualunque segno).
    """
    occ = [[] for _ in range(n_vars)]
    for ci, clause in enumerate(cnf):
        for lit in clause:
            occ[abs(lit) - 1].append(ci)
    return occ


def clause_satisfaction_array(cnf: CNF, assignment: List[bool]) -> List[bool]:
    return [evaluate_clause(c, assignment) for c in cnf]


def breakcount_for_var(var_idx: int, cnf: CNF, assignment: List[bool],
                       clause_sat: List[bool], occ_lists: List[List[int]]) -> int:
    """
    Breakcount: quante clausole attualmente soddisfatte diventano insoddisfatte se flippo var_idx.
    """
    b = 0
    for ci in occ_lists[var_idx]:
        if clause_sat[ci]:
            # Ricontrolla la clausola assumendo il flip
            satisfied_now = False
            for lit in cnf[ci]:
                v = abs(lit) - 1
                val = assignment[v]
                if v == var_idx:
                    val = not val  # flip simulato
                if lit < 0:
                    val = not val
                if val:
                    satisfied_now = True
                    break
            if not satisfied_now:
                b += 1
    return b


def make_random_assignment(n_vars: int) -> List[bool]:
    return [random.choice([True, False]) for _ in range(n_vars)]


def pick_unsat_clause_indices(cnf: CNF, clause_sat: List[bool]) -> List[int]:
    return [i for i, s in enumerate(clause_sat) if not s]


def delta_unsat_if_flip(var_idx: int, cnf: CNF, assignment: List[bool],
                        clause_sat: List[bool], occ_lists: List[List[int]]) -> int:
    """
    Delta nel numero di clausole insoddisfatte se flippo var_idx (negativo è buono).
    """
    delta = 0
    for ci in occ_lists[var_idx]:
        was_sat = clause_sat[ci]
        # valuta con flip
        satisfied_after = False
        for lit in cnf[ci]:
            v = abs(lit) - 1
            val = assignment[v]
            if v == var_idx:
                val = not val
            if lit < 0:
                val = not val
            if val:
                satisfied_after = True
                break
        if was_sat and not satisfied_after:
            delta += 1
        elif not was_sat and satisfied_after:
            delta -= 1
    return delta


def samplesat(cnf: CNF,
              n_vars: int,
              max_flips: int = 1_000_000,
              max_restarts: int = 20,
              p_walk: float = 0.5,
              temperature: float = 0.05,
              cooling: float = 0.9999,
              greediness: float = 0.8,
              seed: Optional[int] = None) -> Tuple[Optional[List[bool]], int]:
    """
    Implementazione pratica di SampleSAT/WalkSAT con criterio di accettazione Metropolis.

    Parametri chiave:
    - p_walk: probabilità di fare un flip casuale in una clausola insoddisfatta.
    - greediness: quando non camminiamo a caso, con questa probabilità scegliamo la variabile con breakcount minimo,
      altrimenti ne scegliamo una a caso tra quelle della clausola (esplorazione).
    - temperature & cooling: controllo dell'accettazione di mosse peggiorative (simulated annealing).
    """
    if seed is not None:
        random.seed(seed)

    occ_lists = build_occurrence_lists(cnf, n_vars)

    total_flips = 0
    for _ in range(max_restarts):
        assignment = make_random_assignment(n_vars)
        clause_sat = clause_satisfaction_array(cnf, assignment)
        T = temperature

        if all(clause_sat):
            return assignment, total_flips

        for _ in range(max_flips):
            total_flips += 1
            unsat_indices = pick_unsat_clause_indices(cnf, clause_sat)
            if not unsat_indices:
                return assignment, total_flips

            ci = random.choice(unsat_indices)
            clause = cnf[ci]

            # Modalità walk vs. scelta guidata
            if random.random() < p_walk:
                var_idx = abs(random.choice(clause)) - 1
            else:
                # con probabilità "greediness" scegli il breakcount minimo, altrimenti esplora
                if random.random() < greediness:
                    # calcola breakcount su tutte le variabili della clausola
                    bc_pairs = []
                    for lit in clause:
                        v = abs(lit) - 1
                        bc = breakcount_for_var(v, cnf, assignment, clause_sat, occ_lists)
                        bc_pairs.append((bc, v))
                    min_bc = min(bc for bc, _ in bc_pairs)
                    candidates = [v for bc, v in bc_pairs if bc == min_bc]
                    var_idx = random.choice(candidates)
                else:
                    var_idx = abs(random.choice(clause)) - 1

            # Valuta delta insoddisfatte
            d_unsat = delta_unsat_if_flip(var_idx, cnf, assignment, clause_sat, occ_lists)

            # Accetta flip: sempre se migliora; se peggiora, accetta con prob. Metropolis
            accept = (d_unsat < 0) or (random.random() < math.exp(-d_unsat / max(T, 1e-12)))

            if accept:
                # Applica flip e aggiorna stato delle clausole toccate
                assignment[var_idx] = not assignment[var_idx]
                for cj in occ_lists[var_idx]:
                    clause_sat[cj] = evaluate_clause(cnf[cj], assignment)

            if all(clause_sat):
                return assignment, total_flips

            # Raffreddamento
            T *= cooling

    return None, total_flips


In [21]:
# Esempio: (x1 ∨ ¬x2) ∧ (¬x1 ∨ x3) ∧ (x2 ∨ x3)
cnf = [
    [1, -2],
    [-1, 3],
    [2, 3],
]
n_vars = 3

solution, flips = samplesat(
    cnf, n_vars,
    max_flips=100000,
    max_restarts=10,
    p_walk=0.5,
    temperature=0.1,
    cooling=0.9995,
    greediness=0.9,
    seed=42
)

if solution is not None:
    print("Soddisfacibile. Assegnazione:", solution, "Flips:", flips)
else:
    print("Non trovata soluzione nei limiti. Flips eseguiti:", flips)


Soddisfacibile. Assegnazione: [True, True, True] Flips: 3


In [22]:
# CNF: (x1 ∨ ¬x2) ∧ (¬x1 ∨ x3) ∧ (x2 ∨ x3)
cnf = [
    [1, -2],   # x1 OR ¬x2
    [-1, 3],   # ¬x1 OR x3
    [2, 3]     # x2 OR x3
]
n_vars = 3


In [23]:
# Assumendo che la funzione samplesat sia già definita come nel codice precedente

solution, flips = samplesat(
    cnf, n_vars,
    max_flips=100000,
    max_restarts=10,
    p_walk=0.5,
    temperature=0.1,
    cooling=0.9995,
    greediness=0.9,
    seed=42
)

if solution is not None:
    print("✅ Soddisfacibile!")
    print("Assegnazione trovata:", solution)
    print("Numero di flip effettuati:", flips)
else:
    print("❌ Nessuna soluzione trovata nei limiti.")


✅ Soddisfacibile!
Assegnazione trovata: [True, True, True]
Numero di flip effettuati: 3


In [24]:
def test_samplesat_multiple_runs(cnf, n_vars, runs=20):
    found_solutions = []
    for i in range(runs):
        sol, flips = samplesat(
            cnf, n_vars,
            max_flips=100000,
            max_restarts=10,
            p_walk=0.5,
            temperature=0.1,
            cooling=0.9995,
            greediness=0.9
        )
        if sol is not None:
            found_solutions.append(sol)
            print(f"Run {i+1}: ✅ soluzione trovata in {flips} flip -> {sol}")
        else:
            print(f"Run {i+1}: ❌ nessuna soluzione trovata")

    print("\nTotale soluzioni trovate:", len(found_solutions), "su", runs)
    if found_solutions:
        # Rimuove duplicati
        unique_solutions = {tuple(s) for s in found_solutions}
        print("Soluzioni uniche trovate:", unique_solutions)

# Esempio con la formula di prima
cnf = [
    [1, -2],
    [-1, 3],
    [2, 3]
]
test_samplesat_multiple_runs(cnf, 3, runs=100)


Run 1: ✅ soluzione trovata in 1 flip -> [True, False, True]
Run 2: ✅ soluzione trovata in 0 flip -> [False, False, True]
Run 3: ✅ soluzione trovata in 0 flip -> [True, False, True]
Run 4: ✅ soluzione trovata in 0 flip -> [True, False, True]
Run 5: ✅ soluzione trovata in 1 flip -> [False, False, True]
Run 6: ✅ soluzione trovata in 2 flip -> [False, False, True]
Run 7: ✅ soluzione trovata in 1 flip -> [False, False, True]
Run 8: ✅ soluzione trovata in 0 flip -> [True, False, True]
Run 9: ✅ soluzione trovata in 0 flip -> [True, True, True]
Run 10: ✅ soluzione trovata in 1 flip -> [False, False, True]
Run 11: ✅ soluzione trovata in 5 flip -> [False, False, True]
Run 12: ✅ soluzione trovata in 0 flip -> [True, True, True]
Run 13: ✅ soluzione trovata in 2 flip -> [False, False, True]
Run 14: ✅ soluzione trovata in 2 flip -> [True, True, True]
Run 15: ✅ soluzione trovata in 1 flip -> [True, True, True]
Run 16: ✅ soluzione trovata in 0 flip -> [False, False, True]
Run 17: ✅ soluzione trovata i

In [25]:
import itertools
import random
import math
from typing import List, Tuple, Optional

Clause = List[int]
CNF = List[Clause]

# --- Funzioni di utilità per SAT ---
def evaluate_clause(clause: Clause, assignment: List[bool]) -> bool:
    for lit in clause:
        var = abs(lit) - 1
        val = assignment[var]
        if lit < 0:
            val = not val
        if val:
            return True
    return False

def clause_satisfaction_array(cnf: CNF, assignment: List[bool]) -> List[bool]:
    return [evaluate_clause(c, assignment) for c in cnf]

def pick_unsat_clause_indices(cnf: CNF, clause_sat: List[bool]) -> List[int]:
    return [i for i, s in enumerate(clause_sat) if not s]

def build_occurrence_lists(cnf: CNF, n_vars: int) -> List[List[int]]:
    occ = [[] for _ in range(n_vars)]
    for ci, clause in enumerate(cnf):
        for lit in clause:
            occ[abs(lit) - 1].append(ci)
    return occ

def breakcount_for_var(var_idx: int, cnf: CNF, assignment: List[bool],
                       clause_sat: List[bool], occ_lists: List[List[int]]) -> int:
    b = 0
    for ci in occ_lists[var_idx]:
        if clause_sat[ci]:
            satisfied_now = False
            for lit in cnf[ci]:
                v = abs(lit) - 1
                val = assignment[v]
                if v == var_idx:
                    val = not val
                if lit < 0:
                    val = not val
                if val:
                    satisfied_now = True
                    break
            if not satisfied_now:
                b += 1
    return b

def delta_unsat_if_flip(var_idx: int, cnf: CNF, assignment: List[bool],
                        clause_sat: List[bool], occ_lists: List[List[int]]) -> int:
    delta = 0
    for ci in occ_lists[var_idx]:
        was_sat = clause_sat[ci]
        satisfied_after = False
        for lit in cnf[ci]:
            v = abs(lit) - 1
            val = assignment[v]
            if v == var_idx:
                val = not val
            if lit < 0:
                val = not val
            if val:
                satisfied_after = True
                break
        if was_sat and not satisfied_after:
            delta += 1
        elif not was_sat and satisfied_after:
            delta -= 1
    return delta

def make_random_assignment(n_vars: int) -> List[bool]:
    return [random.choice([True, False]) for _ in range(n_vars)]

# --- Implementazione SampleSAT ---
def samplesat(cnf: CNF,
              n_vars: int,
              max_flips: int = 10000,
              max_restarts: int = 10,
              p_walk: float = 0.5,
              temperature: float = 0.05,
              cooling: float = 0.9999,
              greediness: float = 0.8) -> Optional[List[bool]]:

    occ_lists = build_occurrence_lists(cnf, n_vars)

    for _ in range(max_restarts):
        assignment = make_random_assignment(n_vars)
        clause_sat = clause_satisfaction_array(cnf, assignment)
        T = temperature

        if all(clause_sat):
            return assignment

        for _ in range(max_flips):
            unsat_indices = pick_unsat_clause_indices(cnf, clause_sat)
            if not unsat_indices:
                return assignment

            ci = random.choice(unsat_indices)
            clause = cnf[ci]

            if random.random() < p_walk:
                var_idx = abs(random.choice(clause)) - 1
            else:
                if random.random() < greediness:
                    bc_pairs = [(breakcount_for_var(abs(lit)-1, cnf, assignment, clause_sat, occ_lists),
                                 abs(lit)-1) for lit in clause]
                    min_bc = min(bc for bc, _ in bc_pairs)
                    candidates = [v for bc, v in bc_pairs if bc == min_bc]
                    var_idx = random.choice(candidates)
                else:
                    var_idx = abs(random.choice(clause)) - 1

            d_unsat = delta_unsat_if_flip(var_idx, cnf, assignment, clause_sat, occ_lists)
            accept = (d_unsat < 0) or (random.random() < math.exp(-d_unsat / max(T, 1e-12)))

            if accept:
                assignment[var_idx] = not assignment[var_idx]
                for cj in occ_lists[var_idx]:
                    clause_sat[cj] = evaluate_clause(cnf[cj], assignment)

            if all(clause_sat):
                return assignment

            T *= cooling

    return None

# --- Conteggio esatto ---
def exact_count(cnf: CNF, n_vars: int) -> int:
    count = 0
    for bits in itertools.product([False, True], repeat=n_vars):
        if all(evaluate_clause(c, bits) for c in cnf):
            count += 1
    return count

# --- Stima #SAT con SampleSAT ---
def estimate_count_samplesat(cnf: CNF, n_vars: int, trials: int = 1000) -> int:
    success = 0
    for _ in range(trials):
        sol = samplesat(cnf, n_vars)
        if sol is not None:
            success += 1
    prob_sat = success / trials
    return int(prob_sat * (2 ** n_vars))

# --- Esempio ---
if __name__ == "__main__":
    cnf = [
        [1, -2],
        [-1, 3],
        [2, 3]
    ]
    n_vars = 3

    exact = exact_count(cnf, n_vars)
    estimates = estimate_count_samplesat(cnf, n_vars, trials=200000)

    print(f"Conteggio esatto: {exact}")
    print(f"Stima con SampleSAT: {estimates}")


Conteggio esatto: 3
Stima con SampleSAT: 8


In [27]:
import statistics

exact = exact_count(cnf, n_vars)
abs_errors = [abs(e - exact) for e in estimates]
rel_errors = [abs(e - exact) / exact for e in estimates]

print("Exact count:", exact)
print("Mean estimate:", statistics.mean(estimates))
print("Mean absolute error:", statistics.mean(abs_errors))
print("Mean relative error:", statistics.mean(rel_errors))
print("Std deviation of estimates:", statistics.stdev(estimates))


TypeError: 'int' object is not iterable

In [ ]:
import matplotlib.pyplot as plt

plt.hist(estimates, bins=10, edgecolor='black')
plt.axvline(exact, color='red', linestyle='--', label='Exact')
plt.xlabel('Estimated #SAT')
plt.ylabel('Frequency')
plt.legend()
plt.show()
